In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("imputation_outlier_encoding_alldone.csv")  # Load your dataset

In [3]:
#df_clean = df_clean.iloc[0:1]

In [4]:
df.head()

,Alloy,Finish,Length,Weight,Tolerances,GD&T,Order Quantity,LME price,Customer Category,Lead Time (weeks),Quote Price (SEK),Quote Date,Profile Name
0,3.0,5.0,19.4,1.266,0.175,0.0,155702,3.76,1.0,8.0,1.98,2025-05-01,3.045256
1,4.0,6.0,34.7,1.729,0.104,2.0,107945,3.48,2.0,7.0,2.83,2025-09-10,3.136125
2,1.0,5.0,35.0,1.415,0.187,2.0,133490,2.97,1.0,11.0,2.37,2025-04-03,3.012713
3,4.0,5.0,27.5,0.952,0.135,2.0,132768,2.60,0.0,2.0,2.39,2025-09-13,2.960553
4,4.0,4.0,15.9,1.453,0.077,1.0,58805,2.74,3.0,2.0,3.69,2025-09-11,3.043105


In [10]:
df['Quote Date'] = pd.to_datetime(df['Quote Date'])
df = df.sort_values('Quote Date')

# Profile complexity
df['Profile_Complexity'] = (
    df['Weight'] / df['Length'] *
    df['Tolerances'] *
    (df['GD&T'] + 1)
)

# Manufacturing difficulty
df['Manufacturing_Difficulty'] = df['Tolerances'] * (df['GD&T'] + 1)

# LME moving average and lag
df['LME_MA_7'] = df['LME price'].rolling(window=7, min_periods=1).mean()
df['LME_Lag_1'] = df['LME price'].shift(1).bfill()



In [11]:
df.head(5)

,Alloy,Finish,Length,Weight,Tolerances,GD&T,Order Quantity,LME price,Customer Category,Lead Time (weeks),Quote Price (SEK),Quote Date,Profile Name,Profile_Complexity,Manufacturing_Difficulty,LME_MA_7,LME_Lag_1
821,0.0,4.0,17.6,1.718,0.147,0.0,50862,2.75,2.0,11.0,3.83,2025-01-01,3.014848,0.014349,0.147,2.750000,2.75
599,1.0,3.0,27.0,1.245,0.067,2.0,116471,2.96,1.0,9.0,2.67,2025-01-01,3.026308,0.009268,0.201,2.855000,2.75
489,2.0,4.0,24.1,0.885,0.099,2.0,63929,3.97,2.0,7.0,3.60,2025-01-02,2.951505,0.010906,0.297,3.226667,2.96
750,2.0,4.0,26.1,1.042,0.152,1.0,120204,2.83,3.0,10.0,2.61,2025-01-02,3.160254,0.012137,0.304,3.127500,3.97
924,2.0,3.0,34.8,1.368,0.118,0.0,94419,3.59,1.0,2.0,3.06,2025-01-03,2.987982,0.004639,0.118,3.220000,2.83


In [12]:
df.Profile_Complexity.unique()

array([0.0143492 , 0.00926833, 0.01090643, 0.0121367 , 0.00463862,
       0.01805375, 0.01795472, 0.01830514, 0.0151096 , 0.00534433,
       0.01484029, 0.01681448, 0.00828583, 0.02745   , 0.00791839,
       0.00814633, 0.01851068, 0.0296374 , 0.01039366, 0.05103127,
       0.03178676, 0.02244964, 0.01484293, 0.0058503 , 0.01233415,
       0.00551867, 0.00494315, 0.0080841 , 0.00395338, 0.01368055,
       0.00646993, 0.04273054, 0.00524774, 0.01522773, 0.02559447,
       0.00887276, 0.01471965, 0.00905485, 0.01773137, 0.02563649,
       0.01290455, 0.00870339, 0.020163  , 0.00280781, 0.0065076 ,
       0.00197864, 0.01554   , 0.00421677, 0.016104  , 0.01396265,
       0.01709714, 0.00977291, 0.02789761, 0.00844947, 0.0133824 ,
       0.01102537, 0.00684024, 0.02134509, 0.01945679, 0.00454696,
       0.00950741, 0.00413119, 0.00754093, 0.01968   , 0.00627644,
       0.00589565, 0.00233497, 0.01416667, 0.016966  , 0.00657437,
       0.03146633, 0.00854685, 0.00843094, 0.00618333, 0.01242

In [ ]:
output_file = "final2.csv"
df.to_csv(output_file, index=False)